# Projet Huffman

1. [Introduction](#intro)
2. [Fréquence](#frq)
3. [Classe Arbre](#p1)
4. [Algorithme de création d'un arbre de Huffman](#2)
5. [Compression et Décompression](#3)
6. [Fonctions principales](#4)
5. [Passage en ASCII](#4,5)
7. [Pour aller plus loin](#5)
7. [Conclusion](#ccl)

## Introduction <a name="intro"></a>

Ce projet tient à recréer le système d'encodage proposé par David Albert Huffman en 1952. Le principe est décrit dans notre cours. Pour les personnes extrérieures, voici l'intruction de la page [Wikipédia](https://fr.wikipedia.org/wiki/Codage_de_Huffman) à ce sujet : 

>Le codage de Huffman est un algorithme de compression de données sans perte. Le codage de Huffman utilise un code à longueur variable pour représenter un symbole de la source (par exemple un caractère dans un fichier). Le code est déterminé à partir d'une estimation des probabilités d'apparition des symboles de source, un code court étant associé aux symboles de source les plus fréquents.
>
>Un code de Huffman est optimal au sens de la plus courte longueur pour un codage par symbole, et une distribution de probabilité connue. Des méthodes plus complexes réalisant une modélisation probabiliste de la source permettent d'obtenir de meilleurs ratios de compression. 

### Fréquence des caractères <a name="frq"></a>

Ou plutôt nombre d'apparitions. En effet, le système commence par calculer, pour chaque caractère, le nombre de fois qu'il apparaît dans le texte. Nous allons utiliser pour cela un dictionnaire, avec pour clé le caractère et comme valeur son nombre d'apparitions dans le texte.

In [334]:
def dico_apparition_par_charactere(texte):
    """
    string -> dict
    Retourne le dictionnaire du nombre d'apparitions de chaque caractère.
    """
    # Dictionnaire de départ
    dict_apparition = {}
    
    # Pour chaque caractère...
    for caractère in texte:
        
        # ...on regarde s'il est dans le dictionnaire. 
        if caractère in dict_apparition:
            # Si oui, on augmente sa valeur
            dict_apparition[caractère] += 1
        else:
            # Si non, on l'initialise à 1
            dict_apparition[caractère] = 1
            
    return dict_apparition

assert dico_apparition_par_charactere('') == {}
assert dico_apparition_par_charactere('Hello World') == {'H': 1, 'e': 1, 'l': 3, 'o': 2, ' ': 1, 'W': 1, 'r': 1, 'd': 1}
assert dico_apparition_par_charactere('aaabbbcccddd') == {'a': 3, 'b' : 3, 'c': 3, 'd': 3}
assert dico_apparition_par_charactere('Hello') == {'H': 1, 'e': 1, 'l': 2, 'o': 1}

## Partie 1 : Classe Arbre <a name="p1"></a>

Pour répondre au problème, il nous faut un arbre binaire. Mais ici, la structure de l'arbre diffère de celle du cours. En effet, un arbre de Huffman a pour feuilles les lettres du texte auxquelles est associé leur poids (c'est-à-dire leurs nombres d'occurences). Les autres noeuds parents sont constitués de la somme des poids de leurs enfants.

### Partie théorique : explications

#### Structure

Notre classe Arbre devra donc contenir les attributs suivants :

- Gauche : le fils gauche (ou None)
- Droit : le fils droit (ou None)
- Lettre : Pour une feuille, la lettre représentée ; pour les autres noeuds, la concaténation des lettres de ses enfants (nous reviendrons par la suite sur ce point)
- Poids : Pour une feuille, le nombre d'apparitions de la lettre ; pour les autres noeuds, la somme du poids de ses enfants.

#### Utilisation et définitions

Voyons maintenant comment utiliser notre arbre, avec le schéma ci-dessous qui décompose le texte : 'Hello'.

Pour construire un arbre de Huffman, nous devons d'abord partir des feuilles qui sont les différentes lettres, ici : H, e, l, o. Comme ce sont des feuilles, les noeuds représentant ces lettres ont leurs fils à None, et leur poids est leur nombre d'apparitions (ou d'occurences).

La suite de l'abre est très simple : les deux noeuds aux plus petits poids sont assemblés dans un nouveau noeud. Ici, les deux plus petits noeuds sont le 'e' et le 'H'. Ils sont donc rassemblés en tant que fils gauche (le plus fréquent, mais ici leurs poids sont égaux) et fils droit.

Le nouveau noeud créé a donc ses fils d'assignés. Mais nous ne préciserons pas, lors de sa création, sa lettre et son poids, car ces valeurs seront calculées automatiquement selon l'idée suivante : le poids est calculé en faisant la somme du poids des deux fils ; la lettre se compose de la concaténation des lettres du fils gauche et du fils droit (ce qui nous permettra par la suite de retrouver le chemin des lettres).

Voici un exemple des étapes pour comprendre (toujours avec l'arbre ci-dessous) : 
<ol>
    <li>Les noeuds 'e' et 'H' sont assemblés dans un noeud parent.</li>
    <li>Ce noeud parent se voit attribuer le poids 2 (la somme des poids de 'e' et 'H').</li>
    <li>Le noeud parent concatène les lettres de ses fils, et sa lettre devient : 'eH'.</li>
    <li>On répète ce processus jusqu'à avoir un seul noeud.</li>
</ol>

#### Exemple : arbre du mot 'Hello'

![Arbre Huffman Exemple](arbre_exemple_Hello.png)

### Partie pratique : implémentation en python

Nous pouvons maintenant créer notre classe python `Arbre`. Le paramètre l (pour lettre) et p (pour poids) auront une valeur par défaut. Ainsi, nous les renseignerons s'il s'agit d'une feuille, autrement ils seront calculés automatiquement.

Voici la classe `Arbre` :

In [335]:
class Arbre:
    
    # Le construceur : 
    # En paramètre (nom complet) : gauche, droite, lettre ('' si non renseignée), poids (-1 si non renseigné)
    # Les attributs sont les mêmes qu'énnoncés plus haut.
    def __init__(self, g, d,  l='', p=-1):
        """
        Création de l'Arbre.
        """
        self.gauche = g
        self.lettre = l
        self.droit = d
        self.poids = p
        
        # Si le poids vaut -1, alors il n'a pas été renseigné, il faut donc le calculer.
        if p == -1:
            self.calculer_poids()
            
        # Si la lettre vaut '', alors elle n'a pas été renseignée, il faut donc calculer sa valeur.
        if l == '':
            self.calculer_lettres()

    def fils_gauche(self):
        """
        -> Arbre
        Retourne le fils gauche.
        """
        return self.gauche

    def fils_droit(self):
        """
        -> Arbre
        Retourne le fils droit.
        """
        return self.droit

    def calculer_poids(self):
        """Calcule le poids d'un noeud (pour un noeud non feuille) à partir de la somme du poids de ses fils."""
        self.poids = 0
        if self.gauche is not None:
            self.poids += self.fils_gauche().donne_poids()

        if self.droit is not None:
            self.poids += self.fils_droit().donne_poids()

    def calculer_lettres(self):
        """Calcule la valeur de la lettre du noeud en concaténant les lettres de ses fils."""
        if self.fils_gauche() is not None:
            self.lettre += self.fils_gauche().donnee()
        if self.fils_droit() is not None:
            self.lettre += self.fils_droit().donnee()

    def donne_poids(self):
        """
        -> int
        Retourne le poids du noeud.
        """
        return self.poids

    def donnee(self):
        """
        -> string
        Retourne la lettre du noeud.
        """
        return self.lettre

    def __str__(self):
        """
        -> string
        Retourne une chaîne de caractères pour afficher l'arbre à partir de ce noeud.
        """
        
        # Cas de base pour la récursivité
        if self is None:
            return 'None'
        
        return '(' + str(self.gauche) + ',' + str(self.droit) + ',' + self.lettre + ',' + str(self.poids) + ')'
    
    def __eq__(self, other):
        """
        -> bool
        Retourne vrai si deux arbres sont identiques, faux sinon.
        """
        # Cas de base pour la récursivité
        if self is None and other is None:
            return True
        
        return self.donne_poids() == other.donne_poids() and self.fils_droit() == other.fils_droit() and self.fils_gauche() == other.fils_gauche()

L'arbre de l'exemple 'Hello' sera créé ainsi :

In [336]:
noeud_e = Arbre(None, None, 'e', 1) # Lettre e avec 1 apparition
noeud_o = Arbre(None, None, 'o', 1) # Lettre o avec 1 apparition
noeud_H = Arbre(None, None, 'H', 1) # Lettre H avec 1 apparition
noeud_l = Arbre(None, None, 'l', 2) # Lettre l avec 2 apparitions

# Lors de la création des noeuds parents, on ne renseigne pas la lettre et le poids, 
# car leurs valeurs seront calculées automatiquement dans le constructeur.
noeud_parent_eH = Arbre(noeud_e, noeud_H)
noeud_parent_eHo = Arbre(noeud_parent_eH, noeud_o)
arbre_eHol = Arbre(noeud_parent_eHo, noeud_l)

# On appelle la méthode __str__ pour afficher l'arbre.
print(arbre_eHol)

((((None,None,e,1),(None,None,H,1),eH,2),(None,None,o,1),eHo,3),(None,None,l,2),eHol,5)


## Partie 2 : Algorithme de création d'un arbre de Huffman <a name="2"></a>

#### A. Création d'un tableau d'arbre

Afin de créer notre arbre de Huffman, nous pouvons nous aider d'un tableau de feuilles, c'est-à-dire des noeuds qui représentent les lettres du texte. Il nous suffira alors de parcourir ce tableau d'une certaine façon pour créer progressivement notre arbre.

Pour cela, utilisons notre classe `Arbre` et créons autant de feuilles que de caractères dans le dictionnaire d'apparitions. Nous le faisons ici de manière récursive, en supprimant à chaque fois l'élément le plus fréquent du dictionnaire à l'aide de la function `pop()` définie dans python. 

In [337]:
def creation_liste_arbres(dict_de_lettres):
    """
    dict -> list
    Retourne un tableau d'arbre contenant les lettres du dictionnaire.
    """
    
    # Cas de base pour la récursivité
    if len(dict_de_lettres) == 0:
        return []
    
    # Conversion du dictionnaire en tableau
    else:
        
        # Sélection du premier élément du dictionnaire.
        # Nous utilisons 'list()' pour convertir la liste d'éléments en tableau et pouvoir utiliser l'index '[0]'.
        lettre, poids = list(dict_de_lettres.items())[0]

        # Création de la feuille : 
        # - les fils ont pour valeur None (définition d'une feuille)
        # - la lettre est celle trouvée ci-dessus
        # - le poids est également trouvé ci-dessus
        arbre = Arbre(None, None, lettre, poids)

        # Cet élément du dictionnaire est supprimé, pour effectuer la récursivité.
        dict_de_lettres.pop(lettre)

        # Création de la liste de manière récursive, en rappelant la fonction avec le dictionnaire modifié.
        L = creation_liste_arbres(dict_de_lettres) + [arbre]
        
        # L'élément supprimé ci-dessus est remis dans le dictionnaire pour ne pas le modifier.
        dict_de_lettres[lettre] = poids
        
        # Retour de la liste
        return L

dictionnaire_apparitions_mot_Hello = dico_apparition_par_charactere('Hello')
liste = [Arbre(None, None, 'o',1), Arbre(None, None,'l',2), Arbre(None, None,'e',1), Arbre(None, None,'H',1)]
assert creation_liste_arbres(dictionnaire_apparitions_mot_Hello) == liste

dictionnaire_apparitions_abbccc = dico_apparition_par_charactere('abbccc')
liste_abbccc = [Arbre(None, None, 'c',3), Arbre(None, None,'b',2), Arbre(None, None,'a',1)]
assert creation_liste_arbres(dictionnaire_apparitions_abbccc) == liste_abbccc

#### B. Rechercher le minimum

Pour créer un arbre de Huffman, nous devons d'abord chercher les 2 caractères les plus rares dans le texte. Nous créons donc une fonction `supprime_et_retourne_minimum_tab_arbre` qui cherche l'arbre avec le poids le plus petit dans une liste d'arbres, le supprime et le retourne. Nous retournons également la liste modifiée, qui nous servira par la suite, pour créer notre arbre de Huffman.

In [338]:
def supprime_et_retourne_minimum_tab_arbre(tab_arbre):
    """
    list -> arbre, list
    Retourne l'arbre au plus petit poids ainsi que la liste sans cet arbre.
    """
    
    if len(tab_arbre) == 0:
        return None, []

    # Nous prenons le dernier arbre de la liste en tant que référence, pour pouvoir ensuite le comparer aux autres.
    arbre = tab_arbre[len(tab_arbre)-1]
    
    # L'index nous permettra à la fin de la fonction de supprimer l'arbre trouvé, afin qu'il ne soit plus dans la liste.
    # L'index vaut ici la longeur du tableau moins 1, car nous avons séléctionné le dernier arbre du tableau.
    index_arbre = len(tab_arbre)-1
    
    # Il nous suffit ensuite de comparer l'arbre séléctionné aux autres, pour retourner celui au plus petit poids.
    for i in range(len(tab_arbre)):
        
        # On sélectionne un arbre
        a = tab_arbre[i]
        
        # On compare son poids
        if a.donne_poids() < arbre.donne_poids():
            
            # Puis si cet arbre a un poids inférieur, alors on met à jour notre arbre de référence et notre index
            arbre = a
            index_arbre = i
            
    # Nous supprimons l'arbre du tableau.
    tab_arbre.pop(index_arbre)
    
    # Retour de l'arbre, et du nouveau tableau.
    return arbre, tab_arbre


# Dans notre exemple 'Hello', le dernier élément de la liste est la lettre H.
liste = creation_liste_arbres(dictionnaire_apparitions_mot_Hello)

# Cette lettre est l'une des plus rares, c'est donc celle-ci que l'on va sélectionner et supprimer de la liste.
nouvelle_liste = [Arbre(None, None, 'o',1), Arbre(None, None,'l',2), Arbre(None, None,'e',1)]

# Dans notre exemple 'Hello', 3 lettres ont un poids de 1. 
# L'ordre de la liste peut donc différer. Ainsi, l'assert suivant peut ne pas fonctionner.
# Il est donné à titre d'exemple, pour la compréhension :
# assert supprime_et_retourne_minimum_tab_arbre(liste) == (Arbre(None, None,'H',1), nouvelle_liste)

liste = creation_liste_arbres(dico_apparition_par_charactere('b aaa'))
nouvelle_liste = [Arbre(None, None, 'a', 3), Arbre(None, None, ' ', 1)]
assert supprime_et_retourne_minimum_tab_arbre(liste) == (Arbre(None, None, 'b', 1), nouvelle_liste)

liste = creation_liste_arbres(dico_apparition_par_charactere('A'))
assert supprime_et_retourne_minimum_tab_arbre(liste) == (Arbre(None, None, 'A', 1), [])

#### C. Création de l'arbre de Huffman

Nous avons maintenant tous les éléments nécessaires pour créer notre arbre de Huffman. Le principe de la création se décompose en 3 étapes : 

1. On cherche les deux feuilles qui ont le plus petit poids, renvoyées par deux appels à `supprime_et_retourne_minimal_tab_arbre`
2. On les assemble dans un nouvel arbre (le plus fréquent à gauche)
3. On ajoute ce nouvel arbre à la liste

In [339]:
def creation_arbre_huffman(tab_arbre):
    """
    list -> Arbre
    Retourne un arbre de Huffman à partir d'un tableau d'arbre.
    """
    
    if len(tab_arbre) == 1:
        return tab_arbre[0]
    elif len(tab_arbre) == 0:
        return None
    
    # On effectue les étapes décrites ci-dessus jusqu'à ce qu'il ne reste que 2 arbres dans le tableau.
    while len(tab_arbre) > 2:
        
        # Etape 1 : on cherche les 2 minimums (et on met à jour le tableau)
        arbre1, tab_arbre = supprime_et_retourne_minimum_tab_arbre(tab_arbre)
        arbre2, tab_arbre = supprime_et_retourne_minimum_tab_arbre(tab_arbre)
        
        # Etape 2 : on les assemble dans un arbre avec le plus fréquent à gauche
        _arbre = Arbre(arbre2, arbre1)
        
        # Etape 3 : on ajoute cet arbre à la liste
        tab_arbre.append(_arbre)

    # Il ne reste que deux arbres, on peut alors les assembler dans un arbre que l'on retourne
    fd, tab_arbre = supprime_et_retourne_minimum_tab_arbre(tab_arbre)
    fg, tab_arbre = supprime_et_retourne_minimum_tab_arbre(tab_arbre)
    
    return Arbre(fg, fd)



dictionnaire = dico_apparition_par_charactere('Hello')
liste_arbres = creation_liste_arbres(dictionnaire)
arbre_huffman = Arbre(Arbre(Arbre(Arbre(None, None,'e',1),Arbre(None,None,'H',1),'eH',2), Arbre(None,None,'o',1),'eHo',3),Arbre(None,None,'l',2),'eHol',5)
assert creation_arbre_huffman(liste_arbres) == arbre_huffman

dictionnaire = dico_apparition_par_charactere('b aaa')
liste_arbres = creation_liste_arbres(dictionnaire)
arbre = creation_arbre_huffman(liste_arbres)
assert arbre == Arbre(Arbre(None,None,'a',3), Arbre(Arbre(None,None,' ',1), Arbre(None,None,'b',1),' b',2),'a b',5)

## Partie 3 : Compression et Décompression <a name="3"></a>

### A. Compression

Pour compresser le texte, il faut, à partir de l'arbre, associer à chaque lettre un code. Les codes trouvés seront stockés dans un dictionnaire avec pour clés les lettres associées.

#### 1. Trouvons le chemin

Nous devons donc trouver le chemin (le code) de chaque lettre. Voici le procédé à suivre pour chaque lettre avec notre arbre de Huffman :

- si la lettre est à droit du noeud, on ajoute un '1' au code
- si la lettre est à gauche du noeud, on ajoute un '0'
- on recommence à partir du nouveau noeud (fils droit ou fils gauche) jusqu'à tomber sur une feuille

La difficulté est de savoir algorithmiquement si nous devons aller à droite ou à gauche. Pour nous aider, nous avons créer la méthode `calculer_lettres` dans la classe `Arbre`, qui est appelée pour tous les noeuds non feuilles. Voici les étapes que nous allons suivre à partir de la racine : 

- on regarde dans le fils gauche si dans toutes les lettres il y a celle que nous cherchons
- si elle y est, on ajoute un 0 et on continue dans ce sous arbre
- si elle n'y est pas, on ajoute un 1 et on continue dans le sous arbre droit
- (le cas où la lettre n'est dans aucun des arbres n'existes pas, car nous appelons notre fonction de chemin pour toutes les lettres du texte, qui sont donc dans l'arbre).

In [340]:
def chemin_lettre(l, arbre):
    """
    string, Arbre -> string
    Retourne le chemin de la lettre 'l' dans l'arbre 'arbre' depuis la racine.
    """
    
    #  Si l'arbre est None, nous ne faisons rien car nous avons trouvé la lettre.
    if arbre is not None:
        
        # Initialisation du chemin
        chemin = ''
        
        # On regarde d'abord si le fils gauche existe.
        # Puis on regarde si la lettre cherchée est contenue dans toute les lettres dans ce sous arbre gauche.
        if arbre.fils_gauche() is not None and l in arbre.fils_gauche().donnee():
            # si oui, alors on ajoute 0 au chemin
            chemin += '0'
            # on calcule la suite du chemin récursivement
            chemin_suivant = chemin_lettre(l, arbre.fils_gauche())
            # s'il ne vaut pas None, on l'ajoute
            if chemin_suivant is not None:
                chemin += chemin_suivant
                
        # Si la lettre n'est pas à gauche, on regarde si le fils droit existe.
        # Puis on regarde si la lettre cherchée est contenue dans toute les lettres dans ce sous arbre droit.
        elif arbre.fils_droit() is not None and l in arbre.fils_droit().donnee():
            # si oui, alors on ajoute 1 au chemin
            chemin += '1'
            # on calcule la suite du chemin récursivement
            chemin_suivant = chemin_lettre(l, arbre.fils_droit())
            # s'il ne vaut pas None, on l'ajoute
            if chemin_suivant is not None:
                chemin += chemin_suivant
        
        # on renvoie le chemin
        return chemin

##### Vérification de la fonction `chemin_lettre`

En reprenant le schéma de l'arbre du mot 'Hello' proposé au début de ce notebook, on remarque que pour la lettre 'H', il faut suivre le chemin suivant : gauche, gauche, droite. Ce qui correspond au code '001'. Vérifions cela :

In [341]:
assert chemin_lettre('H', arbre_huffman) == '001'

#### 2. Dictionnaire de codes

Nous pouvons, à partir de la fonction `chemin_lettre` crée un dicitonnaire de codes pour chacune des lettres.

In [342]:
def dico_codes(dico_apparition, arbre_huffman):
    """
    dict, Arbre -> dict
    Retourne un dictionnaire de code à l'aide de l'arbre de Huffman.
    """
    
    # Initialisation du dictionnaire.
    dict_de_codes = {}
    
    # On parcourt toutes les lettres (les clés) pour leur associer un code
    for clé in dico_apparition.keys():
        dict_de_codes[clé] = chemin_lettre(clé, arbre_huffman)
        
    return dict_de_codes


dico_apparition = dico_apparition_par_charactere('Hello')
assert dico_codes(dico_apparition, arbre_huffman) == {'H': '001', 'e': '000', 'l': '1', 'o': '01'}

#### 3. Compressons du texte !

Maintenant que nous avons un code associé à chaque caractère, il nous suffit d'ajouter leur code à une chaîne de caractères, qui représentera le texte compressé.

In [343]:
def compresse_texte(texte, dico_codes):
    """
    string, dict -> string
    Retourne le texte compressé selon le dictionnaire de codes.
    """
    
    # Initialisation de notre texte encodé
    texte_compressé = ''
    
    # Pour chaque caractère, on ajoute son code respectif
    for caractère in texte:
        texte_compressé += dico_codes[caractère]
        
    
    return texte_compressé

##### Vérification de la fonction `compresse_texte`

Reprenons notre arbre donné en exemple, avec le mot 'Hello'. Avec le principe de compression décrit ci-dessus, lettre par lettre, créons le texte compressé.

- H : gauche, gauche, droite -> 001
- e : gauche, gauche, gauche -> 000
- l : droite -> 1
- l : droite -> 1
- 0 : gauche, droite -> 01

Ainsi, nous avons le code : 0010001101 en concaténant tous les codes trouvés.
Vérifions donc notre fonction.

In [344]:
dictionnaire_codes = dico_codes(dico_apparition, arbre_huffman)
assert compresse_texte('Hello', dictionnaire_codes) == '0010001101'


# Selon le même principe, avec le texte 'Hello World'
arbre_huffman_Hello_World = creation_arbre_huffman(creation_liste_arbres(dico_apparition_par_charactere('Hello World')))
dictionnaire_codes_Hello_World = dico_codes(dico_apparition_par_charactere('Hello World'), arbre_huffman_Hello_World)
texte_code_Hello_World = compresse_texte('Hello World', dictionnaire_codes_Hello_World)
assert texte_code_Hello_World == '00010000010100111011100110001101'

### B. Décompression

Pour décompresser le texte, il nous suffit d'effectuer le processus dans le sens inverse : à partir des valeurs du dictionnaire (les codes), concaténer les clés (les caractères).

Les codes n'étant pas séparés pas des espaces, nous devons mettre en place un système qui nous permettent de différencier tous les codes. Pour cela, à chaque nouveau chiffre du code, nous allons regarder s'il est présent dans les valeurs du dictionnaire. Si ce n'est pas le cas, on garde ce chiffre en mémoire, et on y ajoute le suivant. Si c'est le cas, alors on ajoute au texte décompressé la clé, et on remet à blanc nos chiffres gardés en mémoire.

In [345]:
def decompresse_text(texte_compressé, codes):
    """
    string, dict -> string
    Retourne le texte décompressé à partir d'un dictionnaire de codes.
    """
    
    # Initialisation du texte décompressé
    texte_décompressé = ''
    
    # Initialisation du code que l'on va gardé en mémoire
    code = ''
    
    # On boucle sur tous les chiffres du texte compressé
    for n in texte_compressé:
        
        # On ajoute à notre code gardé en mémoire le chiffre
        code += n
        
        # On vérifie si ce code est présent dans les valeurs.
        if code in codes.values():
            
            # Si c'est le cas, on ajoute le caractère correspondant au code :
            # - conversin des clés en tableau
            # - conversion des valeurs en tableau
            # - sélection de la clé qui a le même index que le code
            texte_décompressé += list(codes.keys())[list(codes.values()).index(code)]
            
            # On remet à blanc notre code gardé en mémoire
            code = ''
    
    # On renvoie notre texte décompressé
    return texte_décompressé

assert decompresse_text('0010001101', dictionnaire_codes) == 'Hello'

# Selon le même principe et d'après l'assert de la cellule python précedénte, on obtient :
assert decompresse_text('00010000010100111011100110001101', dictionnaire_codes_Hello_World) == 'Hello World'

## Partie 4 : Fonctions principales <a name="4"></a>

Avec toutes les fonctions précédentes, nous pouvons créer deux nouvelles fonctions principales. L'une compressera un texte et créera un dictionnaire de codes pour le décompresser, et l'autre décompressera le texte compressé à l'aide de ce dictionnaire.

In [346]:
def compresser_un_texte(texte):
    """
    string -> string, dict
    Retourne le texte compressé avec le dictionnaire de codes pour le décompresser.
    """
    
    # Dictionnaire du nombre d'apparitions de chaque caractère dans le texte
    dictionnaire_poids = dico_apparition_par_charactere(texte)
    
    # Création d'un arbre de huffman
    liste_arbres = creation_liste_arbres(dictionnaire_poids)
    arbre_huffman = creation_arbre_huffman(liste_arbres)
    
    # Dicionnaire des codes
    dictionnaire_codes = dico_codes(dictionnaire_poids, arbre_huffman)
    
    # Compression
    texte_compressé = compresse_texte(texte, dictionnaire_codes)
    
    return texte_compressé, dictionnaire_codes

assert compresser_un_texte('Hello') == ('0010001101', {'o': '01', 'l': '1', 'e': '000', 'H': '001'})

In [347]:
def decompresser_un_texte(texte, dictionnaire_codes):
    """
    string, dict -> string
    Retourne le texte décompressé grace au dictionnaire de codes.
    """
    
    # Dictionnaire du nombre d'apparitions de chaque caractère dans le texte
    dictionnaire_poids = dico_apparition_par_charactere(texte)
    
    # Création d'un arbre de huffman
    liste_arbres = creation_liste_arbres(dictionnaire_poids)
    arbre_huffman = creation_arbre_huffman(liste_arbres)
    
    # Décodage
    texte_décompressé = decompresse_text(texte, dictionnaire_codes)
    
    return texte_décompressé


assert decompresser_un_texte('0010001101', {'o': '01', 'l': '1', 'e': '000', 'H': '001'}) == 'Hello'

texte = 'Hello World'
texte_compressé, dictionnaire_codes = compresser_un_texte(texte)

assert decompresser_un_texte(texte_compressé, dictionnaire_codes) == texte

## Partie 4,5 : Passage en ASCII <a name="4,5"></a>

Nous avons encodé le texte. Ainsi, il se retrouve stocké sous la forme de 0 et de 1. Si nous stockions ces données sous cette forme, l'ordinateur encodera tous les bits, un à un, en ASCII. Le texte ainsi codé ne sera que bien plus lourd. 

L'idée est donc de couper notre texte encodé, tous les 7 bits, et stocker non pas ces bits, mais leur forme ASCII. C'est seulement ainsi que nous gagnerons de l'espace.

In [366]:
def encode_ascii(texte_compressé):
    texte_ascii = ''
    
    # Pour couper le texte en 7 bits, il faut peut être rajouter des 0 si 
    # le total n'est pas divisible par 7. On stock le nombre de rajouts.
    bit_à_rajouter = 0
    
    # On boucle sur le texte, avec un pas de 7 car
    # l'encodage ascii est sur 7 bits.
    for i in range(0, len(texte_compressé), 7): 

        # On recupère 7 bits de notre texte encodé
        tmp_data = texte_compressé[i:i+7]
    
        # Pour le dernier tour de boucle, il peut manquer des bits pour
        # que le total soit divisible par 7. On rajoute donc des 0.
        while (len(tmp_data)) % 7 != 0:
            bit_à_rajouter += 1
            tmp_data += '0'
            
        # On convertit les bits en décimal...
        decimal = int(tmp_data, 2)

        # ... pour ensuite utiliser chr() et obtenir la valeur ascii associée.
        # N.B. Le #010b permet de former un str (avec le format) contenant un 
        # entier avec le 0b au début.
        texte_ascii += chr(int(format(decimal, '#010b'), 2))
    
    return texte_ascii, bit_à_rajouter

# N.B. Le surplus se trouve dans le dictionnaire de code, à la clé 'surplus'
texte_compressé, dictionnaire_codes = compresser_un_texte('Hello World! It\'s me')
nouveau_texte_ascii, dictionnaire_codes['surplus'] = encode_ascii(texte_compressé)

#assert nouveau_texte_ascii == '\x08(Z\nm`'
nouveau_texte_ascii

'^\x13h.\x19\x00\x14gDU0'

In [378]:
def decode_ascii(texte_ascii, dict_codes):
    
    # Le binaire codé dans l'ascii
    tmp_bin = ''

    # On boucle sur l'ascii qui contient le binaire.
    for i in range(len(texte_ascii)):
        
        # Le caractère ascii
        c = texte_ascii[i]
        
        # Sa valeur décimale
        dec = ord(c)
        
        # Sa valeur binaire sans le 0b et sans le 0 du début (7 bits oblige)
        binary = format(dec, '#010b')[3:]

        # Si l'on se trouve sur la dernière valeur, il faut enlever le surplus de 0
        # que l'on avait mis pour encoder sur 7 bits.
        if i == len(texte_ascii)-1:
            y = dict_codes['surplus']
            while y > 0:
                binary = binary[0:len(binary)-1]
                y -= 1
        
        # On ajoute la valeur binaire au texte codé
        tmp_bin += binary
    
    # On décode le texte codé
    resultat = decompresser_un_texte(tmp_bin, dict_codes)
    
    return resultat

#assert decode_ascii(nouveau_texte_ascii, dictionnaire_codes) == 'Hello World!'
decode_ascii(nouveau_texte_ascii, dictionnaire_codes)

"Hello World! It's me"

## Partie 5 : Pour aller plus loin <a name="5"></a>

### A. Combien de bits avons nous gagnés ?

Nous avons réussi à compresser et décompresser un texte selon le principe défini par Huffman. Cette méthode permet de réduire la taille d'un fichier. Dans un texte classique, dans la plupart des cas, les caractères sont encodés selon le code ASCII qui utilisent 7 bits par caractère.

Pour nous en rendre compte, nous pouvons créer une fonction `donne_gain` qui renvoie le gain fait grâce à ce nouveau système.

In [379]:
def donne_gain(texte_decompressé, texte_compressé):
    """
    string, string -> int
    Retourne le nombre de bits gagnés grâce à la compression de Huffman.
    """
    
    # Chaque caractère est codé sur 7 bits
    bits_texte_non_compressé = len(texte_decompressé)
    
    # Notre nouveau texte, aussi ascii, mais encodé
    texte_ascii, surplus = encode_ascii(texte_compressé)
    bits_texte_compressé = len(texte_ascii)
    
    # Retour du gain
    return bits_texte_non_compressé - bits_texte_compressé

print('Pour \'Hello\':',donne_gain('Hello', '0010001101'))
print('Pour \'Hello World\':', donne_gain('Hello World', '00010000010100111011100110001101'))

Pour 'Hello': 3
Pour 'Hello World': 6


Nous avons économisé 3 et 6 bits. A grande échelle ce système peut donc se rélever très intéressant.

### B. Depuis un fichier séparé

#### 1. Compresser

Grâce aux deux fonctions principales écites plus haut, il est simple de compresser et décompresser des petits textes. Mais comment faire si nous avons un texte très grand ? Le copier puis le coller peut prendre du temps, et n'est pas très commode dans un fichier python ou un notebook. 

Pour résoudre ce problème, nous pouvons créer une fonction `compresse_depuis_fichier_vers_fichier` qui ira chercher notre fichier texte (situé dans le même dossier), et à partir du texte trouvé, créera un nouveau fichier texte avec le texte compressé ainsi qu'un fichier contenant le dictionnaire de codes pour décompresser.

Pour cela, nous utiliserons la fonction python `open()` avec deux arguments : 

- le premier est le nom du fichier à trouver ou à créer
- le deuxième gère la demande d'accès au fichier. Dans notre cas il vaut soit 'r' (pour 'read') pour lire le fichier, soit 'w+' (pour 'write') pour écrire dans le fichier

De plus, nous devrons utiliser `write()` qui permet d'écrire dans un fichier, ainsi que `close()` pour fermer le fichier une fois que nous aurons terminé de l'utiliser.

Enfin, pour écrire le dictionnaire de code, nous avons besoin de la bibliothèque `json` et sa fonction `dumps()` pour convertir un dict en string.

In [382]:
import json # Importation de la bibliothèque json

def compresse_depuis_fichier_vers_fichier(nom_fichier_source, nom_fichier_destination, nom_fichier_codes):
    """
    string, string, string -> None
    Créer un fichier texte au nom de nom_fichier_destination dans lequel figure le texte compressé. 
    Créer un second fichier texte du nom de nom_fichier_codes où se trouve le dictionnaire de codes pour décompresser.
    """
    
    # On ouvre le fichier avec le texte à compresser pour le lire
    fichier = open(nom_fichier_source, 'r')
    
    # On copie le texte
    texte = fichier.read()
    
    # On ferme le fichier
    fichier.close()
    
    # On compresse le texte
    texte_compressé, dict_codes = compresser_un_texte(texte)
    texte_ascii, dict_codes['surplus'] = encode_ascii(texte_compressé)
    
    # On créer le fichier du texte compressé pour écrire
    fichier_dest = open(nom_fichier_destination, 'w+')
    
    # On écrit dans le fichier le texte compressé
    fichier_dest.write(texte_ascii)
    
    # On ferme ce fichier
    fichier_dest.close()
    
    # On créer le fichier pour stocker le dictionnaire de code pour écrire
    fichier_codes = open(nom_fichier_codes, 'w+')
    
    # On convertit le dictionnaire en string
    dict_en_string = json.dumps(dict_codes)
    
    # On écrit les codes
    fichier_codes.write(dict_en_string)
    # On ferme ce fichier
    fichier_codes.close()
    
    
compresse_depuis_fichier_vers_fichier('texte_source.txt', 'texte_compressé.txt', 'codes.txt')

Pour vérifier cette fonction, il nous suffit de regarder dans le répertoire courant et de s'assurer de la précense des deux nouveaux fichiers qui devrait contenir le texte compressé pour l'un, et le dictionnaire de codes pour l'autre.

#### 2. Décompresser

De la même manière, nous pouvons créer une fontion `décompresse_depuis_fichier_vers_fichier` qui depuis un fichier source du texte compressé et avec un fichier de codes, créer un nouveau fichier avec le texte décompressé.

Cependant, le dictionnaire étant devenu du type string, nous devons réutiliser la bibliothèque `json` et la fonction `loads()` pour convertir du string vers dict.

In [383]:
import json # Importation de la bibliothèque json

def décompresse_depuis_fichier_vers_fichier(nom_fichier_texte_compressé, nom_fichier_codes, nom_fichier_destination):
    """
    stirng, string, string -> None
    Créer un fichier avec le texte décompressé, à partir d'un texte compressé et d'un dictionnaire de codes.
    """
    
    # On ouvre le fichier avec le texte à décompresser pour le lire
    fichier = open(nom_fichier_texte_compressé, 'r')
    # On copie le texte ascii
    texte_ascii = fichier.read()
    # On ferme le fichier
    fichier.close()
    
    # On ouvre le fichier avec les codes pour le lire
    fichier = open(nom_fichier_codes, 'r')
    # On copie le texte
    texte_codes = fichier.read()
    # On ferme le fichier
    fichier.close()
    # On convertit le texte des codes en dictionnaire
    dict_codes = json.loads(texte_codes)
    
    # On créer le fichier pour le texte décompressé
    fichier = open(nom_fichier_destination, 'w+')
    # On y écrit le texte décompressé
    fichier.write(decode_ascii(texte_ascii, dict_codes))
    #fichier.write(decompresser_un_texte(texte_compressé, dict_codes))
    # On ferme le fichier
    fichier.close()
    
décompresse_depuis_fichier_vers_fichier('texte_compressé.txt', 'codes.txt', 'texte_décompressé.txt')

On pourra remarquer un nouveau fichier nommé 'texte_décompressé', qui contient le même texte que le fichier 'texte_source'. Nous avons donc réussi à lire un texte, le coder, puis le décoder.

## Conclusion <a name="ccl"></a>

Nous avons vu comment à partir d'un texte, il est possible de créer un nouveau texte d'une plus petite taille. On dit qu'on a "compressé" le texte.

A noter que pour que ce système marche, il faut donner au décodeur le dictionnaire de codes.

[Dépôt à retrouver ici](https://github.com/4l3x4ndre/Huffman-encoding)